<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
%cd /content/drive/MyDrive/Github/capstone

/content/drive/MyDrive/Github/capstone


In [11]:
#!git pull   #check that git us up to date
#!git checkout -b shaptonstall # create a new branch

#!git push origin shaptonstall #commit changes to my branch

In [12]:
# to commit to github
#!git status # check untracked files
#!git add {filename} # add the files
#!git config --global user.email "sarahapt@umich.edu"
# !git commit

In [7]:
pip install factor-analyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor-analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42564 sha256=29f510b7d34a75e3467f6a7a7122ea711aa3d29e0b706e0b885ade08493e3d63
  Stored in directory: /root/.cache/pip/wheels/24/59/82/6493618e30ed1cb7a013b9e1b0c9e17de80b04dfcef4ba8a4d
Successfully built factor-analyzer


In [78]:
pip install semopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 10.6 MB/s eta 0:00:00
  Created wheel for semopy: filename=semopy-2.3.11-py3-none-any.whl size=1659680 sha256=30f42aaff1265e5225b7a50ad984622718d55438b8a7cd90d9f50084b482d305
  Stored in directory: /root/.cache/pip/wheels/53/ec/0d/0b294c02d8c4e9e80afea58839f2c1b4706770594bc99ec045
Successfully built semopy


Import libraries

In [102]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
from factor_analyzer import Rotator
from factor_analyzer import (ConfirmatoryFactorAnalyzer, ModelSpecificationParser)
from semopy import Model
from semopy import Optimizer
from semopy.inspector import inspect
from semopy import gather_statistics

In [13]:
!ls

 Capstone.ipynb		  README.md			       Webscraping.xlsx
 data_exploration.ipynb  'trivium_full_column_names (1).csv'


In [14]:
# Load all sheets into a dictionary of DataFrames
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)



In [15]:
webscraping.keys()
# webscraping['Events and8']

dict_keys(['Events and8', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Battles and8 Round by Round', 'Battles and8 Traditional', 'Research Questions', 'Notes', 'Notes Columns'])

Events details

In [16]:
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

eventsWDSF = webscraping['Events WDSF'][1:].copy()
new_header = eventsWDSF.iloc[0]
eventsWDSF = eventsWDSF.drop(1)
eventsWDSF.columns = new_header
eventsWDSF.rename(columns={'Name': 'Event'}, inplace=True)
eventsWDSF.drop_duplicates(subset=['Event'], inplace=True)

combined_events = pd.concat([eventsAnd8, eventsWDSF], ignore_index=True)


Threefold Battles

In [17]:
#threefold battles
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
threefold.head(2)

,Event,Category,breaker 1,breaker 2,winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r3j7arti,r3j7inte,r3j8over,r3j8phys,r3j8arti,r3j8inte,r3j9over,r3j9phys,r3j9arti,r3j9inte
0,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Baruzzo,Elpablix,Baruzzo,3,3,2,1,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Yoga,Faby,Faby,2,3,0,2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
pd.merge(threefold, combined_events[['Date', 'Event', 'Format']], on='Event')['Format'].unique()


array([nan], dtype=object)

Trivium Battles

In [19]:
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')


In [20]:
trivium_columns=list(trivium.columns)
exclude=['judge 1 name ', 'judge 2 name', 'judge 3 name', 'judge 4 name', 'judge 5 name', 'judge 6 name ', 'judge 7 name', 'judge 8 name', 'judge 9 name'  ]
trivium_columns= [x for x in trivium_columns if x not in exclude]
trivium=trivium[trivium_columns]
trivium['BattleID']= trivium.index

<ipython-input-20-a75688204c39>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trivium['BattleID']= trivium.index


In [21]:
trivium.shape

(1526, 417)

In [22]:
category_sums_blue = {
    "Creativity": trivium.filter(like="crea").sum(axis=1),
    "Musicality": trivium.filter(like="musi").sum(axis=1),
    "Performance": trivium.filter(like="perf").sum(axis=1),
    "Personality": trivium.filter(like="pers").sum(axis=1),
    "Technique": trivium.filter(like="tech").sum(axis=1),
    "Variety": trivium.filter(like="vari").sum(axis=1),
    "Overall":  trivium.filter(like="over").sum(axis=1)
}

category_sums_red = {
    "Creativity": trivium.filter(like="crea").sum(axis=1)*-1,
    "Musicality": trivium.filter(like="musi").sum(axis=1)*-1,
    "Performance": trivium.filter(like="perf").sum(axis=1)*-1,
    "Personality": trivium.filter(like="pers").sum(axis=1)*-1,
    "Technique": trivium.filter(like="tech").sum(axis=1)*-1,
    "Variety": trivium.filter(like="vari").sum(axis=1)*-1,
    "Overall":  trivium.filter(like="over").sum(axis=1)*-1
}

In [23]:
scorecardBlue=pd.DataFrame({
    "Event": trivium['Event'],
    'Category': trivium['Category'],
    "BattleID": trivium["BattleID"],
    "Side": "BLUE",
    "Breaker": trivium["breaker 2 (blue)"],
        **category_sums_blue,
})

scorecardRed=pd.DataFrame({
    "Event": trivium['Event'],
    'Category': trivium['Category'],
    "BattleID": trivium["BattleID"],
    "Side": "RED",
    "Breaker": trivium["breaker 1 (red)"],
     **category_sums_red,
})

In [24]:
scorecard = pd.concat([scorecardBlue, scorecardRed], ignore_index=True)
scorecard['Score Type']="trivium"
scorecard.shape

(3052, 13)

In [25]:
# scorecard[scorecard['Breaker']=="Ami"].mean(), scorecard[scorecard['Breaker']=="Ami"].count()

In [26]:
# scorecard[scorecard['Breaker']==671].mean(), scorecard[scorecard['Breaker']==671].count()

In [27]:

scorecard.shape

(3052, 13)

In [28]:
scorecard=pd.merge(scorecard, eventsAnd8[['Date', 'Event']], on='Event', how='left')
# today = scorecard['Date'].max()
# total_days = (today - scorecard['Date'].min()).days + 1  # Get the total number of days dynamically
# scorecard['Weight'] = ((today - scorecard['Date']).dt.days / total_days) * 100
# scorecard['Weight'] = 100 - scorecard['Weight']  # Reverse the weights so that newer dates have higher weights

Trivium PCA

In [29]:
# triviumFloat=
triviumScore=scorecardBlue[['Creativity', 'Musicality', 'Performance', 'Personality', 'Technique', 'Variety']]

In [30]:
#imputer also get feature name for plotting later
names=triviumScore.columns
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
triviumNorm = imputer.fit_transform(triviumScore)
features=imputer.get_feature_names_out(input_features= names)


#normalize float columns
triviumNorm= StandardScaler().fit(triviumNorm).transform(triviumNorm)


In [31]:
#set PCA
pca = PCA(n_components=3, random_state=0).fit(triviumNorm)
X_pca = pca.transform(triviumNorm)
X_pca.shape, triviumNorm.shape

((1526, 3), (1526, 6))

In [32]:
features

array(['Creativity', 'Musicality', 'Performance', 'Personality',
       'Technique', 'Variety'], dtype=object)

In [33]:
loadings_abs = np.abs(pca.components_)
pcs = ['PC1', 'PC2', 'PC3']

# Create DataFrame
pca_loadings = pd.DataFrame(loadings_abs.T, columns=pcs, index=features)

# pca_loadings.T.plot.bar(figsize=(10, 20), subplots=True)

In [34]:
print( " factor Analysis accum variance: ",np.cumsum(pca.explained_variance_ratio_)[2])

 factor Analysis accum variance:  0.8822364614685656


In [35]:
pca_loadings.style.background_gradient()

,PC1,PC2,PC3
Creativity,0.362888,0.143561,0.713032
Musicality,0.403575,0.450374,0.269483
Performance,0.469467,0.239068,0.327888
Personality,0.409212,0.452774,0.295638
Technique,0.350112,0.704047,0.190013
Variety,0.441596,0.136793,0.433527


**Factor Analysis.**
Kaiser-Meyer-Olkin (KMO) Test measures the suitability of data for factor analysis. It determines the adequacy for each observed variable and for the complete model. KMO estimates the proportion of variance among all the observed variable. Lower proportion id more suitable for factor analysis. KMO values range between 0 and 1. Value of KMO less than 0.6 is considered inadequate.

In [36]:
kmo_all,kmo_model=calculate_kmo(triviumNorm)
kmo_model

0.6828153227522676

In [38]:
# perform factor analysis
fa = FactorAnalyzer(3, rotation=None)
fa.fit(triviumNorm)
X_transformed = fa.fit_transform(triviumNorm)

In [39]:
print( " factor Analysis accum variance: ",fa.get_factor_variance()[2][-1], fa.get_params())

 factor Analysis accum variance:  0.7885636265232234 {'bounds': (0.005, 1), 'impute': 'median', 'is_corr_matrix': False, 'method': 'minres', 'n_factors': 3, 'rotation': None, 'rotation_kwargs': {}, 'svd_method': 'randomized', 'use_smc': True}


In [40]:
faLoadings=np.abs(fa.loadings_)
fa = ['Factor1', 'Factor2', 'Factor3']

# Create DataFrame
fa_loadings = pd.DataFrame(faLoadings, columns=pcs, index=features)

# fa_loadings.T.plot.bar(figsize=(10, 20), subplots=True)


In [41]:
fa_loadings.style.background_gradient()

,PC1,PC2,PC3
Creativity,0.584749,0.144549,0.356931
Musicality,0.669645,0.352689,0.148181
Performance,0.850926,0.149023,0.278467
Personality,0.728653,0.465250,0.282656
Technique,0.682145,0.707579,0.173523
Variety,0.840725,0.059744,0.537386


In [42]:
pca_loadings.style.background_gradient()

,PC1,PC2,PC3
Creativity,0.362888,0.143561,0.713032
Musicality,0.403575,0.450374,0.269483
Performance,0.469467,0.239068,0.327888
Personality,0.409212,0.452774,0.295638
Technique,0.350112,0.704047,0.190013
Variety,0.441596,0.136793,0.433527


In [43]:
normdf = pd.DataFrame(triviumNorm, columns=features)
normdf

,Creativity,Musicality,Performance,Personality,Technique,Variety
0,-1.516892,-0.136318,0.560938,0.313784,2.168559,-1.087127
1,0.104730,0.941751,1.159945,1.219928,1.537056,0.823927
2,-0.986180,-0.407644,0.994701,0.134033,1.989348,1.752689
3,-0.204852,-1.808894,-2.155249,0.202978,-1.040159,-2.638185
4,2.301292,1.279402,-1.318704,-0.110971,-1.620459,1.233439
...,...,...,...,...,...,...
1521,0.709153,0.523306,0.612576,0.514465,0.530065,0.216351
1522,0.576475,-0.313585,0.602249,0.168505,0.231381,-0.140968
1523,1.490480,-0.024170,0.808803,1.334427,-0.118506,0.145423
1524,1.136672,0.859751,0.437005,0.563712,0.350854,0.554934


In [65]:
model_dict = {"Physical": ['Technique', 'Variety'], "Artistic": ['Creativity', 'Personality'], "Interpretive": ['Performance', 'Musicality']}


model_spec = ModelSpecificationParser.parse_model_specification_from_dict(normdf, model_dict)

cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)


In [66]:
cfa.fit(normdf.values)

cfa.loadings_

array([[0.46116327, 0.        , 0.        ],
       [0.79510733, 0.        , 0.        ],
       [0.        , 0.98885383, 0.        ],
       [0.        , 0.63190673, 0.        ],
       [0.        , 0.        , 0.8877747 ],
       [0.        , 0.        , 0.5921204 ]])

In [ ]:
model = '''
    F1 =~ var1 + var2 + var3
    F2 =~ var4 + var5 + var6
'''

In [82]:
model_spec = '''Physical =~ Technique + Variety
Artistic =~ Creativity + Personality
Interpretive =~ Performance + Musicality
'''

In [85]:
modelCFA= Model(model_spec)


In [100]:
modelCFA.fit(normdf)

SolverResult(fun=1.0002738219571627, success=True, n_it=34, x=array([0.70333331, 2.20873099, 0.7049806 , 0.18421626, 0.13451218,
       0.81585685, 0.79908718, 0.30858884, 0.70211414, 0.60280773,
       0.20108571, 0.10155843, 0.74745609, 0.25253753, 0.63024941]), message='Optimization terminated successfully', name_method='SLSQP', name_obj='MLW')

In [101]:
modelCFA.inspect()

,lval,op,rval,Estimate,Std. Err,z-value,p-value
0,Technique,~,Physical,1.000000,-,-,-
1,Variety,~,Physical,0.703333,0.029187,24.097385,0.0
2,Creativity,~,Artistic,1.000000,-,-,-
3,Personality,~,Artistic,2.208731,0.152256,14.506704,0.0
4,Performance,~,Interpretive,1.000000,-,-,-
5,Musicality,~,Interpretive,0.704981,0.024745,28.489786,0.0
6,Artistic,~~,Artistic,0.184216,0.022579,8.158696,0.0
7,Artistic,~~,Physical,0.134512,0.014929,9.009963,0.0
8,Interpretive,~~,Interpretive,0.799087,0.038265,20.883249,0.0
9,Interpretive,~~,Artistic,0.308589,0.025005,12.341045,0.0


1. **Technique ~ Physical**: The estimated regression coefficient between the latent variable Technique and the observed variable Physical is 1.000, indicating a perfect relationship.
2. **Variety ~ Physical**: The estimated regression coefficient between the latent variable Variety and the observed variable Physical is 0.703. This suggests that Physical has a significant positive effect on Variety.
3. **Creativity ~ Artistic**: The estimated regression coefficient between the latent variable Creativity and the latent variable Artistic is 1.000, indicating a perfect relationship.
4. **Personality ~ Artistic**: The estimated regression coefficient between the latent variable Personality and the latent variable Artistic is 2.209. This indicates a significant positive effect of Artistic on Personality.
5. **Performance ~ Interpretive**: The estimated regression coefficient between the latent variable Performance and the latent variable Interpretive is 1.000, indicating a perfect relationship.
6. **Musicality ~ Interpretive**: The estimated regression coefficient between the latent variable Musicality and the latent variable Interpretive is 0.705. This suggests that Interpretive has a significant positive effect on Musicality.
